# Desarrollo y exploracion de las bases de datos

Link a la carpeta donde se está trabajando:
https://drive.google.com/drive/folders/1T2RLe3RsoYE_DBRdEyJCqbEXx4lDr44A?usp=sharing

In [ ]:
!pip install scikit-learn
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.2 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29880 sha256=81b019ea331bab0949cab5368fcbcd75db58d68e160a1b287fb0c6102435d51f
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=04969989c9699a1341bed1247e4fc4aecc23139b86458a309ae6bfbf312023ab
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


## 0. Preparación

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

### 0.1. Preparacion base de datos 2023-2024

#### 0.1.1. Generalización

**¿Qué se hizo?**
\
\
A partir de las bases de datos otorgadas, se escogieron 3 para el desarrollo del proyecto.
- Datos historicos 2020-2021
- Datos historicos 2022-2023
- Datos historicos 2023-2024

Las primeras dos bases de datos, están distribuidas de manera en el que se muestra la produccion por hora durante los años mencionados de los contaminantes que procesa cada estación, en el que cada hoja corresponde a una zona diferente.

En este caso, de manera manual se decidió crear un archivo .csv, el cual es un archivo por zona, siguiendo el nombre de {zona}_añoinicial_añofinal.csv.

Los datos historicos de 2023-2024 cuentan con una distribución diferente, en el que los datos de todas las zonas se encuentran presentes dentro de una misma hoja, al igual que ahora los encabezados son diferentes, donde en lugar de ser una sola fila con el nombre de la molecula, son tres filas, con el nombre de fila, nombre de la molecula y unidad de medición respectivamente.

De manera un poco ineficiente, pero favoreciendo el entendimiento humano de los datos, se intentó separar de manera manual, sin embargo esto no fue posible debido a la magitud de la informacion encontrada en la misma hoja.
Por ende se realizó el siguiente codigo, el cual, después de haber eliminado la columna de zona y unidad de medición d emanera manual, estandariza el formato de la información acual con la de años anteriores, al igual que se considera eficiente si se decide continuar con este formato en años futuros.

#### 0.1.2 Codigo de estandarización de formato en base de datos Datos Historicos 2023-2024

In [ ]:
import pandas as pd

# Ruta del archivo
archivo_principal = '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/TODO/DATOS_HISTÓRICOS_2023_2024_TODAS_ESTACIONES_ITESM.csv'

# Nombres de las zonas y sus respectivos archivos
zonas = ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'NORESTE2', 'SURESTE2', 'SUROESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE3', 'NOROESTE3']

# Columnas que tiene cada conjunto
columnas_por_zona = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Lee el archivo principal (usando low_memory=False y dtype=str para evitar advertencias)
df = pd.read_csv(archivo_principal, header=0, index_col=False, low_memory=False, dtype=str)

# Ubicación donde se guardarán los archivos
ruta_guardado = '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2023_2024'

# Procesa cada conjunto de columnas
for i, zona in enumerate(zonas):
    # Determina las columnas para esta zona (se asume que hay una columna vacía entre cada conjunto de datos)
    inicio = i * (len(columnas_por_zona) + 1) + 1  # Se incluye la columna vacía como separación
    fin = inicio + len(columnas_por_zona)

    # Extrae la columna "date" y el conjunto de columnas para la zona
    columnas_zona = ['date'] + list(df.columns[inicio:fin])

    # Filtra el DataFrame por esas columnas
    df_zona = df[columnas_zona]

    # Renombra las columnas para que no incluyan los sufijos numéricos
    df_zona.columns = ['date'] + columnas_por_zona

    # Guarda el archivo CSV para cada zona
    nombre_archivo = f"{zona}_2023_2024.csv"
    df_zona.to_csv(ruta_guardado + nombre_archivo, index=False)

print("Archivos generados correctamente.")


Archivos generados correctamente.


### 0.2. Unión de archivos .csv por año

Una vez compilado el código anterior, se tienen archivos .csv de cada partícula divididos por año, para contar con un codigo más eficiente, se decide unir todos los codigos en un solo archivo, el cual una los valores del 2020 al 2024.

Es importante tomar en cuenta que este archivo solo une la información, pero los datos siguen sin procesar, es decir, todos los valores vacíos y nulos siguien presentes en los datasets.

In [ ]:
import os
import pandas as pd

# Rutas de las carpetas con archivos CSV
folders = [
    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2020_2021/',
    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2022_2023/',
    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2023_2024/'
]

# Ruta de destino para el archivo combinado
output_folder = '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/'
output_file = os.path.join(output_folder, 'BASES_2020_2024_SP.csv')

# Crear una lista para almacenar los DataFrames
df_list = []

# Leer y concatenar los archivos CSV de cada carpeta
for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder, filename)
            # Leer el archivo CSV
            df = pd.read_csv(file_path)

            # Extraer la zona del nombre del archivo
            zona = filename.split('_')[0]  # Obtiene el primer segmento del nombre (por ejemplo, 'CENTRO')

            # Agregar la nueva columna de zona al DataFrame
            df['zona'] = zona

            # Agregar el DataFrame a la lista
            df_list.append(df)

# Concatenar todos los DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

# Guardar el archivo combinado en la nueva ubicación
combined_df.to_csv(output_file, index=False)

print(f'Archivos combinados y guardados en: {output_file}')


<ipython-input-5-ce31e149fcb9>:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(df_list, ignore_index=True)


Archivos combinados y guardados en: /content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP.csv


## 1. Limpieza y Exploracion de Datos

### 1.0 REBOOT

In [ ]:
# [REBOOT]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Cargar los DataFrames
df_2020_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP.csv')
df_UBI = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/UBICACIONES/UBI.csv')

<ipython-input-9-c242efa54de3>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP.csv')


### 1.1. Limpieza

#### 1.1.1 Asegurar formato punto decimal

In [ ]:
import pandas as pd

# Suponiendo que ya tienes tu dataframe df_2020_2024 cargado

# Lista de columnas donde se debe reemplazar la coma por punto
columnas_a_reemplazar = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Reemplazar comas por puntos en las columnas seleccionadas
df_2020_2024[columnas_a_reemplazar] = df_2020_2024[columnas_a_reemplazar].replace({',': '.'}, regex=True)

# Convertir las columnas a tipo numérico
df_2020_2024[columnas_a_reemplazar] = df_2020_2024[columnas_a_reemplazar].apply(pd.to_numeric, errors='coerce')

# Verificar los cambios
print(df_2020_2024[columnas_a_reemplazar].head())

     CO   NO   NO2   NOX    O3    PM10  PM2.5    PRS  RAINF    RH  SO2     SR  \
0   NaN  NaN   NaN   NaN   NaN   86.34  60.91    NaN    NaN   NaN  NaN  0.000   
1   NaN  NaN   6.5   9.8  19.0  112.01  85.64  713.6    0.0  91.0  NaN  0.158   
2   NaN  NaN   5.6   8.8  18.0  100.01  72.39  712.8    0.0  91.0  NaN  0.156   
3  3.22  3.2   7.2  10.4  14.0  106.20  70.25  712.4    0.0  92.0  2.9  0.158   
4  3.26  4.7  10.6  15.3   5.0  141.86  93.72  712.0    0.0  92.0  3.3  0.163   

    TOUT  WSR    WDR  
0    NaN  NaN    NaN  
1  10.49  3.7    2.0  
2  10.51  1.9  144.0  
3  10.64  2.7   28.0  
4  10.73  2.0   31.0  


#### 1.1.2. Separación de fecha y hora

In [ ]:

# Función para dividir la columna 'date' en 'date' y 'time'
def split_date_time(df):
    # Convertir la columna 'date' a tipo datetime
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y %H:%M')

    # Extraer la fecha y la hora en columnas separadas
    df['time'] = df['date'].dt.time   # Extraer la hora
    df['date'] = df['date'].dt.date   # Extraer la fecha

    return df

# Aplicar la función a cada DataFrame

df_2020_2024 = split_date_time(df_2020_2024)

# Guardar los DataFrames con las nuevas columnas

#df_2020_2024.to_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP_mod.csv', index=False)

print("Las columnas 'date' y 'time' han sido separadas y los archivos guardados.")

Las columnas 'date' y 'time' han sido separadas y los archivos guardados.


#### 1.1.3. Encoding de las zonas

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Función para codificar la columna 'Zona' y generar el diccionario de encoding
def encode_zona(df):
    # Crear el codificador
    le = LabelEncoder()

    # Ajustar el codificador a la columna 'Zona' y transformar los valores
    df['zona_encoded'] = le.fit_transform(df['zona'])

    # Crear el diccionario de encoding
    zona_dict = dict(zip(le.classes_, le.transform(le.classes_)))

    # Eliminar la columna 'Zona' original
    df = df.drop(columns=['zona'])

    return df, zona_dict

# Aplicar la codificación a cada DataFrame
df_2020_2024, zona_dict_2020_2024 = encode_zona(df_2020_2024)

# Mostrar los diccionarios de encoding para cada DataFrame
print("Diccionario de encoding para 2020-2024:", zona_dict_2020_2024)

# Guardar los DataFrames con la columna 'Zona' eliminada y la columna 'Zona_encoded'
#df_2020_2024.to_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP_encoded.csv', index=False)

print("Los archivos con la columna 'Zona' eliminada han sido guardados.")


Diccionario de encoding para 2020-2024: {'CENTRO': 0, 'NORESTE': 1, 'NORESTE2': 2, 'NORESTE3': 3, 'NOROESTE': 4, 'NOROESTE2': 5, 'NOROESTE3': 6, 'NORTE': 7, 'NORTE2': 8, 'SUR': 9, 'SURESTE': 10, 'SURESTE2': 11, 'SURESTE3': 12, 'SUROESTE': 13, 'SUROESTE2': 14}
Los archivos con la columna 'Zona' eliminada han sido guardados.


#### 1.1.4. Asegurar propiedades numericas

In [ ]:
# Definir columnas relevantes para la imputación
columnas_relevantes = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Convertir solo las columnas relevantes a formato numérico
df_2020_2024[columnas_relevantes] = df_2020_2024[columnas_relevantes].apply(pd.to_numeric, errors='coerce')

### 1.2. Exploración

#### 1.2.1 Dimensión del dataset

In [ ]:
import pandas as pd
# Obtener la dimensión del dataset
dim_dataset = df_2020_2024.shape
print("Cantidad de registros:", dim_dataset[0])
print("Cantidad de columnas:", dim_dataset[1])


Cantidad de registros: 659361
Cantidad de columnas: 18


#### 1.2.2. Cantidad de datos nulos

In [ ]:
import pandas as pd
# 1.1.2. Cantidad de datos nulos
nulos = df_2020_2024.isnull().sum()
print("\nCantidad de datos nulos por columna:")
print(nulos[nulos > 0])

# Procesar cada dataframe
for df, name in zip([df_2020_2024], ['2020_2024']):
    # Calcular el porcentaje de valores vacíos o "NULL"
    null_percentage = df.isnull().mean() * 100
    print(f"Porcentaje de datos vacíos en TODO{name}:\n", null_percentage)

# df_2020_2021, df_2022_2023, df_2023_2024 son los nombres de los dataframes procesados
# 1.1.3. Descripción de las variables




Cantidad de datos nulos por columna:
CO        86861
NO       113553
NO2      118483
NOX      114007
O3       109258
PM10      35980
PM2.5    160358
PRS       37768
RAINF     36398
RH        68658
SO2      107752
SR        25392
TOUT      35885
WSR       55915
WDR       66289
dtype: int64
Porcentaje de datos vacíos en TODO2020_2024:
 date             0.000000
CO              13.173512
NO              17.221674
NO2             17.969367
NOX             17.290528
O3              16.570285
PM10             5.456798
PM2.5           24.320213
PRS              5.727970
RAINF            5.520193
RH              10.412809
SO2             16.341883
SR               3.851001
TOUT             5.442390
WSR              8.480180
WDR             10.053522
time             0.000000
zona_encoded     0.000000
dtype: float64


In [ ]:

variables_info = []

for column in df_2020_2024.columns:
    var_info = {
        'Nombre': column,
        'Descripción': df_2020_2024[column].describe(include='all').to_dict(),
        'Tipo': df_2020_2024[column].dtype,
        'Valores Nulos': df_2020_2024[column].isnull().sum(),
    }

    # Solo incluir el tipo de variable para los valores posibles
    if isinstance(df_2020_2024[column].dtype, pd.CategoricalDtype) or df_2020_2024[column].dtype == 'object':
        var_info['Valores Posibles'] = 'String'
    elif pd.api.types.is_numeric_dtype(df_2020_2024[column]):
        var_info['Valores Posibles'] = 'Float/Int'
    elif pd.api.types.is_datetime64_any_dtype(df_2020_2024[column]):
        var_info['Valores Posibles'] = 'Datetime'
    else:
        var_info['Valores Posibles'] = 'Otro'

    variables_info.append(var_info)

# Convertir a DataFrame para mejor visualización
variables_df = pd.DataFrame(variables_info)

# Mostrar el DataFrame con la información de las variables
print("\nInformación de las variables:")
print(variables_df)


Información de las variables:
          Nombre                                        Descripción     Tipo  \
0           date  {'count': 659361, 'unique': 1674, 'top': 2023-...   object   
1             CO  {'count': 572500.0, 'mean': 1.3860415772925763...  float64   
2             NO  {'count': 545808.0, 'mean': 11.939413383457188...  float64   
3            NO2  {'count': 540878.0, 'mean': 14.573269092103954...  float64   
4            NOX  {'count': 545354.0, 'mean': 26.287490199026685...  float64   
5             O3  {'count': 550103.0, 'mean': 27.020323793907686...  float64   
6           PM10  {'count': 623381.0, 'mean': 59.87437008827668,...  float64   
7          PM2.5  {'count': 499003.0, 'mean': 20.955216221144966...  float64   
8            PRS  {'count': 621593.0, 'mean': 715.1072130799415,...  float64   
9          RAINF  {'count': 622963.0, 'mean': 0.0188741706971361...  float64   
10            RH  {'count': 590703.0, 'mean': 56.300160283594295...  float64   
11       

In [ ]:
df_2020_2024

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,time,zona_encoded
0,2020-01-01,NaN,NaN,NaN,NaN,NaN,86.34,60.91,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,00:00:00,0
1,2020-01-01,NaN,NaN,6.5,9.8,19.0,112.01,85.64,713.6,0.0,91.0,NaN,0.158,10.49,3.7,2.0,01:00:00,0
2,2020-01-01,NaN,NaN,5.6,8.8,18.0,100.01,72.39,712.8,0.0,91.0,NaN,0.156,10.51,1.9,144.0,02:00:00,0
3,2020-01-01,3.22,3.2,7.2,10.4,14.0,106.20,70.25,712.4,0.0,92.0,2.9,0.158,10.64,2.7,28.0,03:00:00,0
4,2020-01-01,3.26,4.7,10.6,15.3,5.0,141.86,93.72,712.0,0.0,92.0,3.3,0.163,10.73,2.0,31.0,04:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659356,2024-07-31,1.18,4.2,11.1,15.2,16.0,117.00,13.73,707.5,0.0,46.0,2.8,0.003,29.87,13.1,63.0,19:00:00,6
659357,2024-07-31,1.33,4.4,12.5,16.8,13.0,90.00,13.05,707.9,0.0,48.0,2.7,0.000,28.93,11.8,41.0,20:00:00,6
659358,2024-07-31,1.47,4.6,12.1,16.6,12.0,90.00,12.72,708.4,0.0,50.0,3.0,0.000,28.31,11.6,41.0,21:00:00,6
659359,2024-07-31,1.26,4.8,11.3,16.0,12.0,104.00,13.63,709.1,0.0,58.0,3.0,0.000,27.25,15.3,35.0,22:00:00,6


#### 1.2.3 Estadistica General

In [ ]:
# Filtrar las columnas 'date' y 'time' en un nuevo dataframe
df_datetime = df_2020_2024[['date', 'time']].copy()

# Convertir la columna 'date' al formato datetime en df_datetime
df_datetime['date'] = pd.to_datetime(df_datetime['date'])

# Convertir la columna 'time' al formato datetime (sin fecha) para extraer la hora
# Ajustar el formato a '%H:%M:%S' si incluye segundos
df_datetime['time'] = pd.to_datetime(df_datetime['time'], format='%H:%M:%S', errors='coerce').dt.time

# Mostrar el rango (mínimo y máximo) para la columna 'date' y la columna 'time'
rango_fechas = df_datetime.agg({'date': [min, max], 'time': [min, max]})

# Estadísticas para columnas numéricas (excluyendo datetime)
df_numerico = df_2020_2024.select_dtypes(include=[np.number])
estadisticas_numericas = df_numerico.describe()

# Medidas de tendencia central y dispersión para columnas numéricas
tendencia_central = {
    'media': df_numerico.mean(),
    'mediana': df_numerico.median(),
    'moda': df_numerico.mode().iloc[0],  # La primera moda en caso de varias
    'varianza': df_numerico.var(),
    'desviación estándar': df_numerico.std(),
    'rango': df_numerico.max() - df_numerico.min()
}

# Medidas de posición no-central (cuartiles)
cuartiles = df_numerico.quantile([0.25, 0.5, 0.75])

# Impresión del rango de la columna 'date' y 'time'
print("Rango de la columna 'date' y 'time':")
print(rango_fechas)

# Impresión de las estadísticas numéricas
print("\nEstadísticas para columnas numéricas:")
print(estadisticas_numericas)

print("\nMedidas de tendencia central y dispersión:")
for key, value in tendencia_central.items():
    print(f"\n{key.capitalize()}:\n{value}")

print("\nCuartiles:")
print(cuartiles)


<ipython-input-18-c22c346ba649>:12: FutureWarning: The provided callable <built-in function min> is currently using Series.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  rango_fechas = df_datetime.agg({'date': [min, max], 'time': [min, max]})
<ipython-input-18-c22c346ba649>:12: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  rango_fechas = df_datetime.agg({'date': [min, max], 'time': [min, max]})


Rango de la columna 'date' y 'time':
          date      time
min 2020-01-01  00:00:00
max 2024-07-31  23:00:00

Estadísticas para columnas numéricas:
                  CO             NO            NO2            NOX  \
count  572500.000000  545808.000000  540878.000000  545354.000000   
mean        1.386042      11.939413      14.573269      26.287490   
std         0.866079      21.930888      17.867913      28.553123   
min        -0.130000       0.300000   -9999.000000       0.500000   
25%         0.710000       3.100000       6.600000      10.900000   
50%         1.260000       5.000000      11.400000      17.400000   
75%         1.870000      11.000000      19.400000      30.700000   
max        37.000000     945.100000     188.600000     971.800000   

                  O3           PM10          PM2.5            PRS  \
count  550103.000000  623381.000000  499003.000000  621593.000000   
mean       27.020324      59.874370      20.955216     715.107213   
std        18.497419

1.2.3. Calidad de los datos

Esto fue realizado obteniendo la cantidad de outliers dentro de cada variable

**¿Qué se hizo con los outliers?**
En el siguiente codigo, los valores que sean determinados como outliers, seran determinados como valores NaN, para posteriormente ser imputados mediente el algoritmo MICE,  esto con la intencion de no perder infromación que podrian representar, al igual que permitir una mejor normalización de la información.

In [ ]:
import pandas as pd

# Función para eliminar los outliers usando el rango intercuartílico (IQR)
def eliminar_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Contar outliers
    outliers_count = ((col < lower_bound) | (col > upper_bound)).sum()

    # Reemplazar outliers por NaN
    col_cleaned = col.where((col >= lower_bound) & (col <= upper_bound))

    return col_cleaned, outliers_count

# Inicializar un DataFrame para almacenar la cantidad de outliers
outliers_info = {}

# Aplicar la función para eliminar outliers en las columnas relevantes
for col in columnas_relevantes:
    df_2020_2024[col], count = eliminar_outliers(df_2020_2024[col])
    outliers_info[col] = count

# Mostrar resultados, asegurando que se mantengan las columnas 'date', 'time', y 'Zona_encoded'
print("Dataframe con outliers reemplazados por NaN:")
print(df_2020_2024)

# Mostrar la cantidad de outliers por columna
print("\nCantidad de outliers reemplazados por columna:")
for column, count in outliers_info.items():
    print(f"{column}: {count} outliers")


Dataframe con outliers reemplazados por NaN:
              date    CO   NO   NO2   NOX    O3    PM10  PM2.5    PRS  RAINF  \
0       2020-01-01   NaN  NaN   NaN   NaN   NaN   86.34    NaN    NaN    NaN   
1       2020-01-01   NaN  NaN   6.5   9.8  19.0  112.01    NaN  713.6    0.0   
2       2020-01-01   NaN  NaN   5.6   8.8  18.0  100.01    NaN  712.8    0.0   
3       2020-01-01  3.22  3.2   7.2  10.4  14.0  106.20    NaN  712.4    0.0   
4       2020-01-01  3.26  4.7  10.6  15.3   5.0     NaN    NaN  712.0    0.0   
...            ...   ...  ...   ...   ...   ...     ...    ...    ...    ...   
659356  2024-07-31  1.18  4.2  11.1  15.2  16.0  117.00  13.73  707.5    0.0   
659357  2024-07-31  1.33  4.4  12.5  16.8  13.0   90.00  13.05  707.9    0.0   
659358  2024-07-31  1.47  4.6  12.1  16.6  12.0   90.00  12.72  708.4    0.0   
659359  2024-07-31  1.26  4.8  11.3  16.0  12.0  104.00  13.63  709.1    0.0   
659360  2024-07-31   NaN  NaN   NaN   NaN   NaN     NaN    NaN    NaN    Na

In [ ]:
# Medidas de posición no-central (cuartiles)
cuartiles = df_numerico.quantile([0.25, 0.5, 0.75])

# Visualización de los datos numéricos
df_numerico.hist(bins=20, figsize=(14, 10))
plt.suptitle('Histogramas de variables numéricas')
plt.show()



NameError: name 'df_numerico' is not defined

In [ ]:
rdf_numerico.plot(kind='box', subplots=True, layout=(6,4 ), figsize=(18, 30), sharex=False, sharey=False)
plt.suptitle('Boxplots de variables numéricas')
plt.show()

NameError: name 'rdf_numerico' is not defined

In [ ]:
# Análisis de correlación y mapa de calor
correlacion = df_numerico.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlacion, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de calor de correlación')
plt.show()

## Imputacion de datos

Utilizando el algoritmo MICE, de la biblioteca *fancyimput*

In [ ]:
!pip install fancyimpute


In [ ]:
df_2020_2024

In [ ]:
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer  # MICE implementation in fancyimpute

# Preprocesamiento de los datos
df_2020_2024['date'] = pd.to_datetime(df_2020_2024['date'])
df_2020_2024.set_index('date', inplace=True)
df_2020_2024.sort_index(inplace=True)

# Convertir a formato numérico las columnas relevantes (importante para imputar correctamente)
df_2020_2024_numeric = df_2020_2024.apply(pd.to_numeric, errors='coerce')

# Mantener la columna "Zona" separada
df_zona = df_2020_2024[['zona_encoded']]  # Guarda la columna 'Zona'

# Seleccionar las columnas relevantes para la imputación (las numéricas)
variables_para_imputar = df_2020_2024_numeric[['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2', 'WSR' ,	'WDR']]  # Añade más columnas si es necesario

# Aplicar el algoritmo MICE para imputación
mice_imputer = IterativeImputer()  # Por defecto usa regresión bayesiana iterativa (JUSTIFICAR)
df_imputed_mice = mice_imputer.fit_transform(variables_para_imputar)

# Convertir de nuevo a dataframe para mantener el formato original
df_imputed_mice = pd.DataFrame(df_imputed_mice, columns=variables_para_imputar.columns, index=variables_para_imputar.index)

# Reintegrar la columna "Zona" al DataFrame imputado
df_final = pd.concat([df_imputed_mice, df_zona], axis=1)

# Mostrar el DataFrame final
print(df_final)


In [ ]:
import pandas as pd

# Ejemplo de tu DataFrame, df_UBI

# Diccionario de encoding de las zonas
zona_dict = {
    'CENTRO': 0,
    'NORESTE': 1,
    'NORESTE2': 2,
    'NORESTE3': 3,
    'NOROESTE': 4,
    'NOROESTE2': 5,
    'NOROESTE3': 6,
    'NORTE': 7,
    'NORTE2': 8,
    'SUR': 9,
    'SURESTE': 10,
    'SURESTE2': 11,
    'SURESTE3': 12,
    'SUROESTE': 13,
    'SUROESTE2': 14,
}



# 1. Reemplazar la columna 'Zona' con los valores del diccionario
df_UBI['Zona'] = df_UBI['Zona'].map(zona_dict)

# 2. Separar la columna 'location' en 'latitud' y 'longitud'
df_UBI[['latitud', 'longitud']] = df_UBI['location'].str.split(',', expand=True)

# 3. Convertir 'latitud' y 'longitud' en formato numérico
df_UBI['latitud'] = pd.to_numeric(df_UBI['latitud'])
df_UBI['longitud'] = pd.to_numeric(df_UBI['longitud'])

# 4. Eliminar la columna original 'location' si ya no es necesaria
df_UBI = df_UBI.drop(columns=['location'])

# Mostrar el DataFrame actualizado
#print(df_UBI)

import pandas as pd

# Asegúrate de que la columna 'date' ya no sea un índice
df_final = df_final.reset_index()  # 'date' vuelve a ser una columna

# Separar la columna 'date' en 'date' y 'time'
df_final['date'] = pd.to_datetime(df_final['date'])  # Convertir a formato datetime si no lo está
df_final['time'] = df_final['date'].dt.time  # Extraer la hora
df_final['date'] = df_final['date'].dt.date  # Extraer solo la fecha

# Unir los dataframes usando la columna 'zona_encoded' como clave
df_completo = df_final.merge(df_UBI, left_on='zona_encoded', right_on='Zona')

# Opcional: Si quieres que 'date' vuelva a ser el índice después del merge
# df_completo = df_completo.set_index('date')

# Mostrar el DataFrame actualizado
print(df_completo)



In [ ]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
from pyproj import Transformer

# Asegúrate de que las columnas 'date' y 'time' estén en el formato adecuado
df_completo['date'] = pd.to_datetime(df_completo['date']).dt.date  # Solo la fecha
#df_completo['time'] = pd.to_datetime(df_completo['time']).dt.time  # Solo la hora

# Fecha y hora específicas para filtrar
fecha_especifica = '2024-07-31'
hora_especifica = '23:00:00'

# Filtrar los datos por fecha y hora separadas
df_filtrado = df_completo[(df_completo['date'].astype(str) == fecha_especifica) &
                          (df_completo['time'].astype(str) == hora_especifica)]

# Agrupar por Zona para calcular las concentraciones promedio de CO, WSR, WDR para esa fecha/hora
coords_zonas = df_filtrado.groupby('Zona').agg({
    'latitud': 'mean',
    'longitud': 'mean',
    'CO': 'mean',
    'WSR': 'mean',
    'WDR': 'mean'
}).reset_index()

# Crear un GeoDataFrame con las coordenadas de las zonas
gdf_zonas = gpd.GeoDataFrame(
    coords_zonas,
    geometry=gpd.points_from_xy(coords_zonas['longitud'], coords_zonas['latitud']),
    crs="EPSG:4326"  # Sistema de coordenadas WGS 84
)

# Transformar a un sistema de coordenadas proyectadas adecuado para añadir el mapa base
gdf_zonas = gdf_zonas.to_crs(epsg=3857)  # Proyección Web Mercator (compatible con contextily)

# Crear una malla de puntos para la interpolación
lon_grid, lat_grid = np.mgrid[
    coords_zonas['longitud'].min():coords_zonas['longitud'].max():100j,
    coords_zonas['latitud'].min():coords_zonas['latitud'].max():100j
]

# Realizar la interpolación
interpolated_CO = griddata(
    (coords_zonas['longitud'], coords_zonas['latitud']),
    coords_zonas['CO'],
    (lon_grid, lat_grid),
    method='cubic'
)

# Ajustar la interpolación de CO según la velocidad del viento
interpolated_CO_adjusted = interpolated_CO * (1 + coords_zonas['WSR'].mean() / max(coords_zonas['WSR']))

# Crear un transformador para convertir de WGS84 (EPSG:4326) a Web Mercator (EPSG:3857)
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

# Transformar las coordenadas de la malla
lon_grid_proj, lat_grid_proj = transformer.transform(lon_grid, lat_grid)

# Crear el mapa base
fig, ax = plt.subplots(figsize=(10, 8))

# Graficar las estaciones
gdf_zonas.plot(ax=ax, color='black', markersize=100, label='Estaciones')

# Añadir el mapa base de contexto con contextily
ctx.add_basemap(ax, crs=gdf_zonas.crs, source=ctx.providers.OpenStreetMap.Mapnik)

# Graficar el mapa de calor utilizando las coordenadas proyectadas
contour = ax.contourf(lon_grid_proj, lat_grid_proj, interpolated_CO_adjusted, cmap='coolwarm', alpha=0.6)

# Agregar una barra de colores
cbar = fig.colorbar(contour, ax=ax, label='Concentración CO ajustada')

# Añadir título y etiquetas
plt.title(f'Distribución de CO ajustada por viento en {fecha_especifica} {hora_especifica}', fontsize=15)
plt.legend()

# Mostrar el gráfico
plt.show()


In [ ]:
df_2020_2024

In [ ]:
df_imputed_mice

In [ ]:
# Graficar los resultados antes y después de la imputación para una columna, por ejemplo 'CO'
plt.plot(df_2020_2024['CO'], label='CO Original')
plt.plot(df_imputed_mice['CO'], label='CO Imputado (MICE)', linestyle='--', alpha = 0.5)
plt.legend()
 # Rango para el eje X
plt.ylim([-30, 180])

plt.show()


In [ ]:
df_imputed_mice

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Supongamos que df_sin_outliers ya está definido y contiene las columnas de interés

# Seleccionar las columnas a estandarizar
columns_to_standardize = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']

# Estandarización (Media 0, Desviación Estándar 1)
scaler = StandardScaler()
df_standardized = df_imputed_mice.copy()  # Hacer una copia para no modificar el original
df_standardized[columns_to_standardize] = scaler.fit_transform(df_imputed_mice[columns_to_standardize])

# Graficar la distribución estandarizada
plt.figure(figsize=(15, 10))
for i, column in enumerate(columns_to_standardize, 1):
    plt.subplot(4, 2, i)  # Cambié a 4 filas y 2 columnas para adaptarse a 8 columnas
    sns.kdeplot(df_standardized[column].dropna(), fill=True)  # Cambiado 'shade=True' por 'fill=True'
    plt.title(f"Distribución Estandarizada: {column}")

plt.tight_layout()
plt.suptitle("Distribución Estandarizada sin Outliers por Columna", fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()


## 3. Transformacion de los datos

Se busca desarrollar un modelo que analice las tendencias en la aparición de smog.

Dado el alto tráfico y la naturaleza industrial de Monterrey, se han considerado las siguientes variables:

**NOx (Óxidos de nitrógeno)**

Razón: Incluye tanto NO como NO2 y es fundamental en la formación de ozono troposférico, un contaminante secundario que afecta la salud y la calidad del aire. Además, contribuye a la formación de PM2.5.
Fuentes: Principalmente proviene del tráfico vehicular y de actividades industriales.

**CO (Monóxido de carbono)**

Razón: Es un contaminante significativo en áreas urbanas, especialmente por el tráfico. Aunque su impacto inmediato en la salud se manifiesta a concentraciones muy altas, también está asociado con problemas respiratorios a largo plazo. Fuentes: Se origina principalmente por la combustión incompleta en vehículos y sistemas de calefacción.

**O3 (Ozono troposférico)**

Razón: Aunque no se emite directamente, se forma a partir de otros contaminantes (como NOx y compuestos orgánicos volátiles) en presencia de luz solar. Es un irritante respiratorio y puede causar problemas de salud.
Condiciones: Se convierte en un problema especialmente en días soleados y cálidos.

**Discretizacion de los datos (binning)**

La discretización puede llevarse a cabo siguiendo las directrices del PRCA,. A partir de ello, se puede crear un conjunto de variables categóricas que, tras nuestro análisis, permitirá desarrollar un sistema de semaforización que indique el riesgo o la probabilidad de aparición de smog. Las moléculas consideradas en el PRCA incluyen PM10, PM2.5, O3, CO, SO2 y NO2.

Es importante destacar que el smog fotoquímico se forma en un entorno urbano por la coexistencia de reactivos y productos, específicamente cuando están presentes óxidos de nitrógeno (NOx), monóxido de carbono (CO) y otros compuestos orgánicos volátiles (COVs) en combinación con la radiación solar.

**Escalamiento y normalizacion**

Se llevó a cabo un escalado y estandarización de los datos debido a las diferencias significativas entre los valores. Este proceso facilita un análisis más sencillo y coherente, permitiendo que todos los datos sean comparados sobre una misma base.

**Construccion de atributos**

En este caso, se recomienda abordar el análisis de dos maneras:

Muchos contaminantes son el resultado de otros. Por ejemplo, el NOx se compone de NO y NO2, y el ozono se forma a partir del NOx.
\
\
Al trabajar con datos temporales, se pueden agrupar por meses, estaciones y horarios, lo que permite realizar comparaciones entre períodos de alta actividad, como las horas pico y las vacaciones.

Por el momento, nos enfocaremos en las variables que representan moléculas, ya que aquellas que dependen de condiciones ambientales tienden a tener un comportamiento inestable.

Variables a considerar: ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']

### Adelanto preeliminar, matriz de correlaciones entre variables.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Seleccionar las columnas de interés
columns_of_interest = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']
df_selected = df_2020_2024[columns_of_interest]

# Reiniciar el índice del dataframe
df_selected.reset_index(drop=True, inplace=True)

# Graficar boxplots
plt.figure(figsize=(15, 10))
# Usar un boxplot para cada contaminante
for i, column in enumerate(columns_of_interest):
    plt.subplot(3, 3, i + 1)  # Crear una tabla de 3 filas y 3 columnas
    sns.boxplot(y=df_selected[column])
    plt.title(f'Boxplot de {column}')
    plt.xlabel('')

plt.tight_layout()  # Ajustar el espaciado entre los subplots
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_2020_2024

# Seleccionar las columnas de interés
columns_of_interest = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']
df_selected = df_2020_2024[columns_of_interest]

# Calcular la matriz de correlación
correlation_matrix = df_selected.corr()

# Graficar la matriz de correlación usando un mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",
            square=True, linewidths=.5, cbar_kws={"shrink": .8})
plt.title("Matriz de Correlaciones entre Contaminantes", fontsize=16)
plt.show()


## ¿Que se desea realizar?

Un modelo que logre ilustrar el comportamiento y aparición del smog sobre el

1.   Elemento de lista
2.   Elemento de lista

area metropolitana de Nuevo Leon

## ¿Qué variables se van a utilizar?

['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']

In [ ]:
#df_2020_2024.to_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASE_A_USAR_2020_2024.csv', index=False)


# Investigacion

In [ ]:
#Dataframes a utilizar

In [ ]:
#Ubicaciones
#df_UBI = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/UBICACIONES/UBI.csv')
df_UBI
#Contaminantes
df_final

In [ ]:
df_UBI

In [ ]:
import pandas as pd

# Ejemplo de tu DataFrame, df_UBI

# Diccionario de encoding de las zonas
zona_dict = {
    'CENTRO': 0,
    'NORESTE': 1,
    'NORESTE2': 2,
    'NORESTE3': 3,
    'NOROESTE': 4,
    'NOROESTE2': 5,
    'NOROESTE3': 6,
    'NORTE': 7,
    'NORTE2': 8,
    'SUR': 9,
    'SURESTE': 10,
    'SURESTE2': 11,
    'SURESTE3': 12,
    'SUROESTE': 13,
    'SUROESTE2': 14,
}



# 1. Reemplazar la columna 'Zona' con los valores del diccionario
df_UBI['Zona'] = df_UBI['Zona'].map(zona_dict)

# 2. Separar la columna 'location' en 'latitud' y 'longitud'
df_UBI[['latitud', 'longitud']] = df_UBI['location'].str.split(',', expand=True)

# 3. Convertir 'latitud' y 'longitud' en formato numérico
df_UBI['latitud'] = pd.to_numeric(df_UBI['latitud'])
df_UBI['longitud'] = pd.to_numeric(df_UBI['longitud'])

# 4. Eliminar la columna original 'location' si ya no es necesaria
df_UBI = df_UBI.drop(columns=['location'])

# Mostrar el DataFrame actualizado
print(df_UBI)


In [ ]:
df_final

In [ ]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

# Crear una función para calcular la distancia entre dos estaciones
def calcular_distancia(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Crear una matriz vacía para las distancias
n = len(df_UBI)
distancias = np.zeros((n, n))

# Calcular las distancias entre todas las estaciones
for i in range(n):
    for j in range(n):
        coord1 = (df_UBI.iloc[i]['latitud'], df_UBI.iloc[i]['longitud'])
        coord2 = (df_UBI.iloc[j]['latitud'], df_UBI.iloc[j]['longitud'])
        distancias[i, j] = calcular_distancia(coord1, coord2)

# Convertir la matriz en un DataFrame para mejor legibilidad
df_distancias = pd.DataFrame(distancias, index=df_UBI['Clave_Estacion'], columns=df_UBI['Clave_Estacion'])

# Mostrar la matriz de distancias
print(df_distancias)


In [ ]:
# Unir los dataframes usando la columna 'zona_encoded' como clave
df_completo = df_final.merge(df_UBI, left_on='zona_encoded', right_on='Zona')

# Seleccionar solo las columnas de contaminantes y ubicación geográfica
contaminantes_y_coords = df_completo[['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2', 'latitud', 'longitud']]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Normalizar los datos (importante para clustering en datos con escalas diferentes)
scaler = StandardScaler()
datos_normalizados = scaler.fit_transform(contaminantes_y_coords)

# Aplicar KMeans para agrupar las estaciones
n_clusters = 4  # Ajusta este valor según lo que quieras explorar
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
df_completo['cluster'] = kmeans.fit_predict(datos_normalizados)


In [ ]:
import geopandas as gpd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Convertir el DataFrame a un GeoDataFrame
gdf = gpd.GeoDataFrame(df_completo,
                        geometry=gpd.points_from_xy(df_completo['longitud'], df_completo['latitud']))

# Lista de contaminantes para los que se creará un mapa
contaminantes = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']

# Crear un mapa para cada contaminante
for contaminante in contaminantes:
    # Normalizar los datos del contaminante específico
    scaler = StandardScaler()
    contaminante_normalizado = scaler.fit_transform(gdf[[contaminante]])

    # Aplicar KMeans para agrupar las estaciones por contaminante
    n_clusters = 4  # Ajusta este valor según lo que quieras explorar
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    gdf[f'cluster_{contaminante}'] = kmeans.fit_predict(contaminante_normalizado)

    # Configuración de colores para los clusters
    colores = ['red', 'blue', 'green', 'purple']

    # Crear la figura del mapa
    fig, ax = plt.subplots(figsize=(10, 10))

    # Dibujar las estaciones con los colores de los clusters
    gdf.plot(column=f'cluster_{contaminante}',
             cmap='Set1',
             ax=ax,
             legend=True,
             legend_kwds={'label': "Clusters",
                          'orientation': "horizontal"},
             markersize=100)

    # Añadir título y etiquetas
    ax.set_title(f'Mapa de Clusters para {contaminante}')
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

    # Mostrar el mapa
    plt.show()
